In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../pyFiles/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

# from configobj import ConfigObj
# from configobj import flatten_errors
# from validate import Validator
    
import logging

# from analysisParameters import loadParameters
# from loadDataE1 import loadSessionDict

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os

init_notebook_mode(connected=True)


In [2]:
# sessionDict = projectTrajectories(sessionDict)
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
import pickle

file = open('../sessionFiles.pickle', 'rb')
allSessions = pickle.load(file)
file.close()

sessionDict = allSessions[0]

In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

In [5]:
# sessionDict['rawCalibUnity']
# sessionDict['rawCalibGaze']

In [6]:
mergedRaw = pd.merge( sessionDict['rawCalibUnity'], sessionDict['rawCalibGaze'].reset_index(), on ='pupilTimestamp',how='outer',sort=True)
interpRaw = mergedRaw.interpolate(method='linear',downcast='infer')
interpRaw

,azimuthWidth,blockNumber,cameraPos_x,cameraPos_y,cameraPos_z,cameraRot_x,cameraRot_y,cameraRot_z,camera_4x4_R0C0,camera_4x4_R0C1,...,gaze_normal0_z,gaze_normal1_x,gaze_normal1_y,gaze_normal1_z,gaze_point_3d_x,gaze_point_3d_y,gaze_point_3d_z,norm_pos_x,norm_pos_y,world_index
0,30,1,-0.440688,1.117208,-1.612361,3.823870,8.453087,1.778171,0.988964,-0.020894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30,1,-0.440720,1.117209,-1.612368,3.823270,8.448709,1.780497,0.988974,-0.020941,...,0.913099,-0.195675,-0.273558,0.941742,-121.567243,-123.735446,433.107010,0.317689,0.747417,12406.0
2,30,1,-0.440753,1.117210,-1.612375,3.822671,8.444332,1.782824,0.988985,-0.020988,...,0.913099,-0.192771,-0.271569,0.942916,-118.094896,-120.432947,423.303246,0.318794,0.746391,12406.0
3,30,1,-0.440785,1.117211,-1.612382,3.822071,8.439954,1.785150,0.988995,-0.021035,...,0.913166,-0.192771,-0.271569,0.942916,-117.935671,-120.075587,422.522518,0.318704,0.746113,12406.0
4,30,1,-0.440807,1.117204,-1.612373,3.823512,8.434866,1.782717,0.989009,-0.020996,...,0.913232,-0.192771,-0.271569,0.942916,-117.776445,-119.718227,421.741790,0.318614,0.745835,12406.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970,30,1,-0.441800,1.120394,-1.617731,3.291677,8.460082,2.041572,0.988792,-0.026795,...,0.972111,0.288180,0.259266,0.921810,75.007336,79.917736,334.069974,0.645834,0.292825,13187.0
2971,30,1,-0.441810,1.120386,-1.617730,3.294682,8.460566,2.040097,0.988792,-0.026761,...,0.972232,0.288180,0.259266,0.921810,75.118174,79.881727,334.465159,0.645877,0.293163,13187.0
2972,30,1,-0.441820,1.120379,-1.617728,3.297688,8.461050,2.038622,0.988791,-0.026728,...,0.972232,0.289510,0.260345,0.921089,74.725214,79.380116,331.457757,0.646430,0.292597,13187.0
2973,30,1,-0.441830,1.120371,-1.617727,3.300693,8.461534,2.037147,0.988791,-0.026694,...,0.972232,0.289439,0.259285,0.921409,74.723831,79.206766,331.564633,0.646380,0.293117,13187.0


In [7]:
def calcAzEl(row,varName):
    
    x=row['{}_x'.format(varName)]
    y=row['{}_y'.format(varName)]
    z=row['{}_z'.format(varName)]
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 

In [8]:
interpRaw['worldVidTimeSec'] = np.array(interpRaw['pupilTimestamp'] - interpRaw['pupilTimestamp'].iloc[0])
interpRaw['worldVidTimeSec'].tail(10)

2965    26.072226
2966    26.076210
2967    26.076532
2968    26.080378
2969    26.084560
2970    26.087518
2971    26.092663
2972    26.096482
2973    26.098138
2974    26.102518
Name: worldVidTimeSec, dtype: float64

In [9]:
# list(df2)

In [10]:
df2 = interpRaw


def calcGazeAzEl(row):
    
    x = row['gaze_point_3d_x'] - row['cameraPos_x']
    y = row['gaze_point_3d_y'] - row['cameraPos_y']
    z = row['gaze_point_3d_z'] - row['cameraPos_z']
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 


azEl = df2.apply(lambda arbitraryRowName: calcGazeAzEl(arbitraryRowName),axis=1)

df2['gazeInWorldCam_az'],df2['gazeInWorldCam_el'] = zip(*azEl)

df2['deltaT'] = df2['worldVidTimeSec'].diff()

df2['velInWorldCam_az'] = df2['gazeInWorldCam_az'].diff() / df2['deltaT']
df2['velInWorldCam_el'] = df2['gazeInWorldCam_el'].diff() / df2['deltaT']
df2['velInWorldCam'] = np.abs(np.sqrt(df2['gazeInWorldCam_az'].diff()**2  + df2['gazeInWorldCam_el'].diff()**2) / df2['deltaT'])

In [11]:
list(df2)

['azimuthWidth',
 'blockNumber',
 'cameraPos_x',
 'cameraPos_y',
 'cameraPos_z',
 'cameraRot_x',
 'cameraRot_y',
 'cameraRot_z',
 'camera_4x4_R0C0',
 'camera_4x4_R0C1',
 'camera_4x4_R0C2',
 'camera_4x4_R0C3',
 'camera_4x4_R1C0',
 'camera_4x4_R1C1',
 'camera_4x4_R1C2',
 'camera_4x4_R1C3',
 'camera_4x4_R2C0',
 'camera_4x4_R2C1',
 'camera_4x4_R2C2',
 'camera_4x4_R2C3',
 'camera_4x4_R3C0',
 'camera_4x4_R3C1',
 'camera_4x4_R3C2',
 'camera_4x4_R3C3',
 'currentTargetName',
 'elevationHeight',
 'fixationTime',
 'frameTime',
 'isHeadFixed',
 'pupilTimestamp',
 'targeLocalPos_x',
 'targeLocalPos_y',
 'targeLocalPos_z',
 'targetDistance',
 'targetPos_x',
 'targetPos_y',
 'targetPos_z',
 'targetRadiusInDegrees',
 'trialNumber',
 'index',
 'base_data',
 'confidence',
 'eye_center0_3d_x',
 'eye_center0_3d_y',
 'eye_center0_3d_z',
 'eye_center1_3d_x',
 'eye_center1_3d_y',
 'eye_center1_3d_z',
 'gaze_normal0_x',
 'gaze_normal0_y',
 'gaze_normal0_z',
 'gaze_normal1_x',
 'gaze_normal1_y',
 'gaze_normal1

In [12]:

def calcTargetAzEl(row):
    
    x = row['targetPos_x'] - row['cameraPos_x']
    y = row['targetPos_y'] - row['cameraPos_y']
    z = row['targetPos_z'] - row['cameraPos_z']
    
    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))
    
    #Note that the .apply requires you return a single data structure, 
    #so a SINGLE tuple is OK, but the seperate values for az/el are not OK.
    
    return (az,el) 

ballAzEl = df2.apply(lambda arbitraryRowName: calcTargetAzEl(arbitraryRowName),axis=1)

df2['ball_az'],df2['ball_el'] = zip(*ballAzEl)
df2['ballVelInWorldCam_az'] = df2['ball_az'].diff() / df2['deltaT']
df2['ballVelInWorldCam_el'] = df2['ball_el'].diff() / df2['deltaT']
df2['ballVelInWorldCam'] = np.abs(np.sqrt(df2['ballVelInWorldCam_az'].diff()**2  + df2['ballVelInWorldCam_el'].diff()**2) / df2['deltaT'])


In [13]:
ballAzEl

0         (-6.76496098849646, 10.427070859582843)
1       (-6.7699384354787675, 10.427156991155035)
2       (-6.7749158817446915, 10.427243124443699)
3        (-6.779893327219108, 10.427329259448877)
4         (-6.784335310185739, 10.42665853623615)
                          ...                    
2970    (24.207000261201483, -18.686530126496564)
2971    (24.207391617485026, -18.690126946421888)
2972     (24.20778298759488, -18.693723762740174)
2973     (24.208174371531737, -18.69732057542307)
2974     (24.208174371531737, -18.69732057542307)
Length: 2975, dtype: object

In [16]:
# yLim = [-100,100]

yLim=[-50,50]
width=800
height=600



startTimeS = df2['worldVidTimeSec'].iloc[0]
endTimeS = df2['worldVidTimeSec'].iloc[-2]

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

traces = []

###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)


traces.append(bEih_el)
traces.append(bEih_az)


eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=-df2['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='markers',
    marker_size = 5,
    opacity = 1)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='markers',
    marker_size = 5,
    opacity = 1)

traces.append(eih_el)
traces.append(eih_az)

layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(
        rangeslider=dict(visible=True))
)


fig = dict(data=traces, layout=layout)

iplot(fig)


Plotting from index: 1 to 2974


In [ ]:


startTimeS = 3
endTimeS = startTimeS + 1*6

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

eih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

eih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


vel = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['velInWorldCam'][startIndex:endIndex],
    name = 'velInWorldCam_az',
    marker_color = colors_idx[0],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


bVel = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ballVelInWorldCam'][startIndex:endIndex],
    name = 'ballVel',
    marker_color = colors_idx[1],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)


####


fig.append_trace(eih_el, row=1, col=1)
fig.append_trace(eih_az, row=1, col=1)
fig.append_trace(vel, row=2, col=1)

fig.append_trace(bEih_el, row=1, col=1)
fig.append_trace(bEih_az, row=1, col=1)
fig.append_trace(bVel, row=2, col=1)



new strategy!
Let's edit the trial info file to focus on the final calibration assessment
Debug the track there, where the az / el should be very predictable, and comparable to a ground truth

In [ ]:
# df2 = interpRaw

# ballAzEl = df2.apply(lambda arbitraryRowName: calcAzEl(arbitraryRowName,'ballPos'),axis=1)
# df2['ball_az'],df2['ball_el'] = zip(*ballAzEl)

# azEl = df2.apply(lambda arbitraryRowName: calcAzEl(arbitraryRowName,'gaze_point_3d'),axis=1)
# df2['gazeInWorldCam_az'],df2['gazeInWorldCam_el'] = zip(*azEl)

# df2['deltaT'] = df2['worldVidTimeSec'].diff()

# df2['velInWorldCam_az'] = df2['gazeInWorldCam_az'].diff() / df2['deltaT']
# df2['velInWorldCam_el'] = df2['gazeInWorldCam_el'].diff() / df2['deltaT']
# df2['velInWorldCam'] = np.abs(np.sqrt(df2['gazeInWorldCam_az'].diff()**2  + df2['gazeInWorldCam_el'].diff()**2) / df2['deltaT'])

# df2['ballVelInWorldCam_az'] = df2['ball_az'].diff() / df2['deltaT']
# df2['ballVelInWorldCam_el'] = df2['ball_el'].diff() / df2['deltaT']
# df2['ballVelInWorldCam'] = np.abs(np.sqrt(df2['ballVelInWorldCam_az'].diff()**2  + df2['ballVelInWorldCam_el'].diff()**2) / df2['deltaT'])